In [ ]:
# !pip install graphviz



In [ ]:
import math
import numpy as np
import math
import matplotlib.pyplot as plt
%matplotlib inline
import torch

In [ ]:
def f(x):
    return 3*x**2 - 4*x + 5

In [ ]:
xs = np.arange(-5, 5, 0.25)
ys = f(xs)

In [ ]:
plt.plot(xs, ys)

In [ ]:
h = 0.000001
x = 2/3
(f(x+h) - f(x))/h


In [ ]:
a = 2.0
b = -3.0
c = 10.0
d = a*b+c
print(d)


In [ ]:
h = 0.0001
a = 2.0
b = -3.0
c = 10.0

d1 = a*b + c
a += h
d2 = a*b + c

print('d1', d1)
print('d2', d2)
print('slope', (d2 - d1)/h)


In [ ]:
class Value:
    def __init__(self, data, _children=(), _op='', label = ''):
        self.data = data
        self.grad = 0.0
        self._backward = lambda: None
        self._prev = set( _children)
        self._op = _op
        self.label = label

    def __repr__(self):
        return f"Value(data={self.data})"

    def __add__(self, other):
        other = other if isinstance(other, Value) else Value(other)
        out = Value(self.data + other.data, (self, other), '+')       
        def _backward():
            self.grad += 1.0 * out.grad
            other.grad += 1.0 * out.grad
        out._backward = _backward
        return out
        
    def __radd__(self, other):
        return self + other
    
    def __mul__(self, other):
        other = other if isinstance(other, Value) else Value(other)
        out = Value(self.data * other.data, (self, other), '*')
        def _backward():
            self.grad += other.data * out.grad
            other.grad += self.data * out.grad
        out._backward = _backward
        return out

    def __pow__(self, other):
        assert isinstance(other, (int, float)), 'currently only supports int and float powers'
        out = Value(self.data**other, (self,), f'**{other}')
        def _backward():
            self.grad += other * (self.data ** (other - 1)) * out.grad
        out._backward = _backward
        return out 

    def __rmul__(self, other):
        return self * other
        
    def __truediv__(self, other):
        return self * other**-1
    
    def __neg__(self):
        return self * -1

    def __sub__(self, other):
        return self + (-other)
    
    def tanh(self):
        x = self.data
        t = (math.exp(2*x) - 1)/(math.exp(2*x) + 1)
        out = Value(t, (self, ), 'tanh')
        
        def _backward():
            self.grad += (1 - t**2) * out.grad
        out._backward = _backward
        
        return out

    def exp(self):
        x = self.data
        out = Value(math.exp(x), (self, ), 'exp')
        
        def _backward():
            self.grad += out.data * out.grad
        out._backward = _backward
        return out
        
    
    def backward(self):
        topo = []
        visited = set()
        def build_topo(v):
            if v not in visited:
                visited.add(v)
                for child in v._prev:
                    build_topo(child)
                topo.append(v)
        build_topo(self)
        
        self.grad = 1.0
        for node in reversed(topo):
            node._backward()

In [ ]:
a = Value(2.0)
b = Value(4.0)
a - b

In [ ]:
# from graphviz import Digraph

# def trace(root):
#     nodes, edges = set(), set()
#     def build(v):
#         if v not in nodes:
#             nodes.add(v)
#             for child in v._prev:
#                 edges.add((child, v))
#                 build(child)
#     build(root)
#     return nodes, edges
    
# def draw_dot(root):
#     dot = Digraph(format = 'svg', graph_attr={'rankdir': 'LR'})  
#     nodes, edges = trace(root)
#     for n in nodes:
#         uid = str(id(n))
#         dot.node(name = uid, label = "{ %s | data %.4f | grad %.4f }" % (n.label, n.data, n.grad), shape='record')
#         if n._op:
#             dot.node(name= uid + n._op, label = n._op)
#             dot.edge(uid + n._op, uid)
#     for n1, n2 in edges:
#         dot.edge(str(id(n1)), str(id(n2)) +n2._op)
        
#     return dot

In [ ]:
# inputs x1, x2:
x1 = Value(2.0, label='x1')
x2 = Value(0.0, label='x2')
# weights ws1, w2:
w1 = Value(-3.0, label='w1')
w2 = Value(1.0, label='w2')
# bias of the neuron:
b = Value(6.88137355870195432 , label='b')
# x1w1 + x2w2 + b:
x1w1 = x1*w1; x1w1.label = 'x1*w1'
x2w2 = x2*w2; x2w2.label = 'x2*w2'
x1w1x2w2 = x1w1 + x2w2; x1w1x2w2.label = 'x1*w1 + x2*w2'
n = x1w1x2w2 + b; n.label = 'n'

o = n.tanh(); o.label = 'output'
o.backward()

In [ ]:
# draw_dot(o)


In [ ]:
# inputs x1, x2:
x1 = Value(2.0, label='x1')
x2 = Value(0.0, label='x2')
# weights ws1, w2:
w1 = Value(-3.0, label='w1')
w2 = Value(1.0, label='w2')
# bias of the neuron:
b = Value(6.88137355870195432 , label='b')
# x1w1 + x2w2 + b:
x1w1 = x1*w1; x1w1.label = 'x1*w1'
x2w2 = x2*w2; x2w2.label = 'x2*w2'
x1w1x2w2 = x1w1 + x2w2; x1w1x2w2.label = 'x1*w1 + x2*w2'
n = x1w1x2w2 + b; n.label = 'n'

# --------- 
e = (2*n).exp()
o = (e-1)/(e+1)
# ----------
o.label = 'output'
o.backward()
# draw_dot(o)

In [ ]:

topo = []
visited = set()
def build_topo(v):
    if v not in visited:
        visited.add(v)
        for child in v._prev:
            build_topo(child)
        topo.append(v)
build_topo(o)

for node in reversed(topo):
    node._backward

In [ ]:
# draw_dot(o)

In [ ]:
a = Value(3.0, label='a')
b = a + a; b.label = 'b'
b.backward()
# draw_dot(b)

In [ ]:
a = Value(-2.0, label = 'a')
b = Value(3.0, label = 'b')
d= a * b; d.label = 'd'
e = a + b; e.label = 'e'
f = d * e; f.label = 'f'

f.backward()

# draw_dot(f)

In [ ]:
import torch


x1 = torch.Tensor([2.0]).double() ; x1.requires_grad=True
x2 = torch.Tensor([0.0]).double() ; x2.requires_grad=True
w1 = torch.Tensor([-3.0]).double() ; w1.requires_grad=True
w2 = torch.Tensor([1.0]).double() ; w2.requires_grad=True
b = torch.Tensor([6.8813735870195432]).double() ; b.requires_grad=True
n = x1*w1 + x2*w2 + b
o = torch.tanh(n)

print(o.data.item())
o.backward()

print('-----')
print('gradients:')
print('x2', x2.grad.item())
print('w2', w2.grad.item())
print('x1', x1.grad.item())
print('w1', w1.grad.item())
print('\n\n')






In [ ]:
torch.Tensor([[1,2,3],[4,5,6]])

In [ ]:
o.item()

In [ ]:
class Neuron:
    
    def __init__(self, nin):
        self.w = [Value(np.random.uniform(-1,1)) for _ in range(nin)]
        self.b = Value(np.random.uniform(-1, 1))
        
    def __call__(self, x):
        # w * x + b
        act = sum((wi*xi for wi, xi in zip(self.w, x)), self.b)
        out = act.tanh()
        return out
        
    def parameters(self):
        return self.w + [self.b]
        # returns parameters list, named the same as pytorch version
        
class Layer:
    def __init__(self, nin, nout):
        self.neurons = [Neuron(nin) for _ in range(nout)]
    def __call__(self, x):
        outs = [n(x) for n in self.neurons]
        return outs[0] if len(outs) == 1 else outs
        
    def parameters(self):
        return [p for neuron in self.neurons for p in neuron.parameters()]

# above list replaces:
    # def parameters(self):
    #     params = []
    #     for neuron in self.neurons:
    #         ps = neuron.parameters()
    #         params.extend(ps)
    #     return params


class MLP:
    def __init__(self, nin, nouts):
        sz = [nin] + nouts
        self.layers = [Layer(sz[i], sz[i+1]) for i in range(len(nouts))]
    def __call__(self, x):
        for layer in self.layers:
            x = layer(x)
        return x
    def parameters(self):
        return [p for layer in self.layers for p in layer.parameters()]



In [ ]:
# init tiny net

x = [2.0, 3.0, -1.0]
n = MLP(3,[4, 4, 1])
n(x), n(x).data


In [ ]:
n.parameters()

In [ ]:
xs= [
    [2.0, 3.0, -1.0],
    [3.0, -1.0, 0.5],
    [0.5, 1.0, 1.0],
    [1.0, 1.0, -1.0],
]

ys = [1.0, -1.0, -1.0, 1.0]


In [ ]:
ypred = [n(x) for x in xs]
loss = sum([(yout - ygt)**2 for ygt, yout in zip(ys, ypred)])
loss

In [ ]:
for p in n.parameters():
    p.data += -0.01 * p.grad

In [ ]:
# y prediction loss:  y ground truth and y output 
# predicted output minus ground truth, squared
#  final loss is sum of losses, ie sum of all (pred-truth)**2 

ypred = [n(x) for x in xs]
loss = sum([(yout - ygt)**2 for ygt, yout in zip(ys, ypred)])
loss

In [ ]:
loss.backward()

In [ ]:
# the effect of this node on the loss

n.layers[0].neurons[0].w[0].grad

In [ ]:
for p in n.parameters():
    p.data += -0.01 * p.grad

In [ ]:
ypred

In [ ]:
n.parameters()


In [ ]:
for p in n.parameters():
    p.data += -0.01 * p.grad

In [ ]:
n.parameters()

In [ ]:
loss.data

    

In [ ]:
for k in range(20):

    # forward
    ypred =[n(x) for x in xs]
    loss = sum((yout -ygt)**2 for ygt, yout in zip(ys, ypred))

    for p in n.parameters():
        p.grad = 0
    loss.backward()
    
    for p in n.parameters():
        p.data += -0.08 * p.grad
    
    print(k, loss.data)


In [ ]:
for k in range(20):

    # forward pass
    ypred =[n(x) for x in xs]
    loss = sum((yout -ygt)**2 for ygt, yout in zip(ys, ypred))

    # backward pass
    for p in n.parameters():
        p.grad = 0
    loss.backward()
    
    for p in n.parameters():
        p.data += -0.08 * p.grad
    
    print(k, loss.data)
